<img src="FondoACET.jpg",width=2000, height=260>

# Taller de Python 2017
## Funciones, condiciones y NumPy

**A partir de los siguientes datos calcular:**
* El número de Reynolds.
* La rugosidad relativa.
* El factor de fricción. 
* La pérdida de carga en metros.

|   Descripción    | Valor | Unidad |
|:----------------:|:-----:|:------:|
| Diámetro nominal | 3     |   $pulg$ |
| Cédula           | 40    |          |
| Rugosidad abs.   | 0.05  |   $mm$   |
| Longitud         | 9.60  |      $m$ |
| Caudal másico    | 1     |   $\frac{kg}{s}$   |
| Fluido           | Ác. sulfúrico  94% |       |
| Temperatura      | 52.78 |     $°C$     |
| Densidad         | 1801.6 | $\frac{kg}{m^3}$  |
| Viscosidad       | 0.01  | $Pa \cdot s$ |


**Las ecuaciones a utilizar son:**

\begin{equation}
N_R = \dfrac{vD\rho}{\mu}\\
\tag*{Número de Reynolds}
\end{equation}

\begin{equation}
f = 64/N_{R}\\
\tag*{Factor para régimen laminar}
\end{equation}

\begin{equation}
f = \dfrac{0.25}{
		\left[log\left(\dfrac{1}{3.7 \cdot (D/\epsilon)}+
		\dfrac{5.74}{N^{0.9}_{R}}\right)\right]^{2}}\\
\tag*{Factor para régimen turbulento}
\end{equation}

\begin{equation}
h_f = f \dfrac{L}{D} \dfrac{v^2}{2g}\\
\tag*{Ecuación de Darcy}
\end{equation}


## Diagrama de flujo

<img src="Flowchart.png",width=2000, height=260>

In [1]:
# Sólo es necesario importar numpy como np para llevar a cabo los cálculos
import numpy as np

In [2]:
'''
BLOQUE DE ECUACIONES

Conviene definir todas aquellas funciones necesarias para realizar las tareas.
'''

# Número de Reynolds

def reynolds(v, D, rho, mu):
    Re_out = v*D*rho / mu
    return Re_out


# Rugosidad relativa

def Rugosidad_relativa(D, Rugosidad_absoluta):
    r_relativa = D/Rugosidad_absoluta
    return r_relativa


# Factor de fricción

def f(Re, r_relativa):
    if Re < 2000:
        f_out = 64/Re
    elif 2000 < Re < 4000:
        raise ValuError('No se puede predecir el comportamiento del fluido')
    else:
        f_out = 0.25/(np.log10((1/(3.7*r_relativa)) + 5.74/(Re**0.9)))**2
    return f_out


# Pérdida de carga debida a la fricción.

def hL(factor, D, L, v):
    g = 9.81 # Ejemplo de hardcoding
    hL_out = factor * (L/D) * (v**2) / (2*g)
    return hL_out

In [3]:
'''
BLOQUE DE VARIABLES

Para realizar los cálculos es necesario asignar valores a las varibales, en este caso podemos utilizar la función input para
escribir un programa que nos sea útil para distintos casos sin la necesidad de modificar el código. 
'''

Datos_1 = str(input('¿Es la velocidad un dato? Sí o No: ')) 

Re = None

if Datos_1.lower() == 'sí':
    v = float(input('Velocidad media del fluido en m/s: '))
    rho = float(input('Densidad en kg/m3: '))
    mu = float(input('Visocsidad dinámica en Pa.s: '))
    D = float(input('Diámetro de la tubería en m: '))
    Re = reynolds(v, D, rho, mu)

elif Datos_1.lower() == 'no':
    Datos_2 = str(input('¿Es el caudal un dato? Sí o No: '))
    if Datos_2.lower() == 'sí':
        Datos_3 = str(input('¿Cuál? volumétrico o másico: '))
        if Datos_3.lower() == 'volumétrico':
            Q = float(input('Caudal volumétrico m3/s: '))
            rho = float(input('Densidad en kg/m3: '))
            mu = float(input('Viscosidad dinámica en Pa.s: '))
            D = float(input('Diámetro de la tubería en m: '))
            As = 3.14 * (D**2) / 4
            v = Q / As
            Re = reynolds(v, D, rho, mu)
        else:
            Q = float(input('Caudal másico kg/s: '))
            rho = float(input('Densidad en kg/m3: '))
            mu = float(input('Viscosidad dinámica en Pa.s: '))
            D = float(input('Diámetro de la tubería en m: '))
            As = 3.14 * (D**2) / 4
            v = Q / (As*rho)
            Re = reynolds(v, D, rho, mu)
    else:
        raise ValueError('Estamos en problemas, te falta un dato')

print('El número de Reynolds es: ', Re)

# Calcular el factor de fricción requiere conocer el número de Reynolds y
# la rugosidad relativa para el material de la tubería.
# La rugosidad relativa se expresa como diámetro/epsilon 
# (Notación del libro de Mott).

Rugosidad_absoluta = float(input('Rugosidad absoluta: '))
Rugosidad_rel = Rugosidad_relativa(D, Rugosidad_absoluta)

print('La rugosidad relativa es: ', Rugosidad_rel)

factor = f(Re, Rugosidad_rel)
print('El factor de fricción es: ', round(factor,5))

¿Es la velocidad un dato? Sí o No: no
¿Es el caudal un dato? Sí o No: sí
¿Cuál? volumétrico o másico: másico
Caudal másico kg/s: 1
Densidad en kg/m3: 1801.6
Viscosidad dinámica en Pa.s: 0.01
Diámetro de la tubería en m: 0.07793
El número de Reynolds es:  1634.6533431521511
Rugosidad absoluta: 0.05e-3
La rugosidad relativa es:  1558.6
El factor de fricción es:  0.039152031999999996


In [4]:
'''
BLOQUE PARA EL CÁLCULO DE LA PÉRDIDA DE CARGA

Todas las variables puestas en juego en la ecuación de Darcy son datos, salvo el factor de fricción calculado en el bloque
anterior.
'''

L = float(input('Longitud de la tubería en m: '))

pérdida = hL(factor, D, L, v)
print('La pérdida de carga expresada en m es: ', '%.3e'%pérdida)

Longitud de la tubería en m: 9.60
La pérdida de carga expresada en m es:  0.003332323040401417
